# ArangoDB 적용

In [1]:
%%capture

!pip install openai langchain python-arango

In [ ]:
# 🔐 API 및 DB 설정
from langchain.chat_models import ChatOpenAI
from arango import ArangoClient
import os, json

os.environ["OPENAI_API_KEY"] = "sk-..."  # 본인의 OpenAI 키 입력

client = ArangoClient()
db = client.db("_system", username="root", password="openSesame")

# 컬렉션 생성
if not db.has_collection("nodes"):
    db.create_collection("nodes")
if not db.has_collection("edges"):
    db.create_collection("edges", edge=True)

# 🧠 LLM 프롬프트 구성
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

llm = ChatOpenAI(temperature=0)

prompt_template = PromptTemplate.from_template("""
너는 지식 그래프 생성 전문가야. 다음 텍스트에서 개체와 관계를 추출하여 JSON 형식으로 반환해.

텍스트:
\"\"\"{text}\"\"\"

출력 형식:
{
  "nodes": [
    {{ "_key": "...", "name": "...", "type": "..." }},
    ...
  ],
  "edges": [
    {{ "_from": "nodes/...", "_to": "nodes/...", "relation": "..." }},
    ...
  ]
}
""")

chain = LLMChain(llm=llm, prompt=prompt_template)

# 📄 입력 예시
input_text = "Steve Jobs는 Apple의 창립자이며 Pixar에도 관여했다. 그는 iPhone 개발을 주도했다."
result = chain.run(text=input_text)

print("🔍 Raw LLM Result:\n", result)

# 🧩 결과 파싱 및 ArangoDB 삽입
parsed = json.loads(result)

nodes_col = db.collection("nodes")
for node in parsed["nodes"]:
    if not nodes_col.has(node["_key"]):
        nodes_col.insert(node)

edges_col = db.collection("edges")
for edge in parsed["edges"]:
    edges_col.insert(edge)

print("✅ 그래프 노드 및 엣지 삽입 완료")

### 1. Named Graph 정의 및 Edge Definition 추가

In [ ]:
# Named Graph 생성
if not db.has_graph("knowledge_graph"):
    graph = db.create_graph("knowledge_graph")
    graph.create_edge_definition(
        edge_collection="edges",
        from_vertex_collections=["nodes"],
        to_vertex_collections=["nodes"]
    )
else:
    graph = db.graph("knowledge_graph")


### 2. AQL로 경로 탐색 (예: steve_jobs 주변 탐색)

In [ ]:
query = """
FOR v, e, p IN 1..2 OUTBOUND 'nodes/steve_jobs' GRAPH 'knowledge_graph'
RETURN {vertex: v, edge: e}
"""

cursor = db.aql.execute(query)
graph_data = list(cursor)
print("🔎 탐색된 노드 및 엣지 수:", len(graph_data))

### 3. Python 시각화 (NetworkX + Matplotlib)

In [ ]:
%%capture

!pip install networkx matplotlib

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

G = nx.DiGraph()

# 정점과 간선을 그래프에 추가
for item in graph_data:
    v = item['vertex']
    e = item['edge']
    from_node = e['_from'].split('/')[1]
    to_node = e['_to'].split('/')[1]
    relation = e.get('relation', '')

    G.add_node(from_node, label=from_node)
    G.add_node(to_node, label=to_node)
    G.add_edge(from_node, to_node, label=relation)

# 그래프 시각화
plt.figure(figsize=(8, 6))
pos = nx.spring_layout(G)

# 노드와 엣지 그리기
nx.draw(G, pos, with_labels=True, node_color="lightblue", node_size=2000, font_size=10)
edge_labels = nx.get_edge_attributes(G, 'label')
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=9)
plt.title("🧠 LLM 기반 Named Graph 시각화")
plt.show()
